In [1]:
!date

Sat Feb 15 15:14:40 UTC 2020


# 削除

作成したリソース情報を読み込む

In [2]:
import json

f = open('res_info.json', 'r')
res_info = json.load(f)
f.close()

res_info

{'vender': 'gcp',
 'cloud_sql_ip': '10.253.0.3',
 'cloud_sql_name': 'topse-wordpress-mysql-20200215112636',
 'cluster_name': 'topse-cluster',
 'dns_name': 'topse.private.',
 'filestore_ip': '10.196.216.218',
 'filestore_name': 'nfs-server',
 'private_zone_name': 'topse-private-zone',
 'region': 'us-west1',
 'subnet_name': 'topse-subnet',
 'vpc_name': 'topse-vpc',
 'vpc_peering_name': 'topse-vpc-peering',
 'zones': ['us-west1-a', 'us-west1-b']}

## kubernatesリソースの削除

In [3]:
!kubectl delete secret mysql
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress-ingress.yaml
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress-service.yaml
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress-backendconfig.yaml
!kubectl delete -f k8s-wordpress-mysql/wordpress/wordpress.yaml

secret "mysql" deleted
ingress.extensions "wordpress-ingress" deleted
service "wordpress" deleted
backendconfig.cloud.google.com "wordpress-backendconfig" deleted
deployment.apps "wordpress" deleted


In [4]:
!kubectl delete -f k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml
!kubectl delete -f k8s-wordpress-mysql/nfs/gcp-filestore.yaml

persistentvolumeclaim "fileserver-claim" deleted
persistentvolume "fileserver" deleted


## Cloud DNS 削除

In [5]:
!gcloud dns record-sets transaction start --zone={res_info['private_zone_name']}
!gcloud dns record-sets transaction remove {res_info['cloud_sql_ip']} --name=mysql.{res_info['dns_name']} --ttl=300 --type=A --zone={res_info['private_zone_name']}
!gcloud dns record-sets transaction remove {res_info['filestore_ip']} --name=nfs.{res_info['dns_name']} --ttl=300 --type=A --zone={res_info['private_zone_name']}
!gcloud dns record-sets transaction execute --zone={res_info['private_zone_name']}
!gcloud beta dns managed-zones delete {res_info['private_zone_name']}

Transaction started [transaction.yaml].
Record removal appended to transaction at [transaction.yaml].
Record removal appended to transaction at [transaction.yaml].
Executed transaction [transaction.yaml] for managed-zone [topse-private-zone].
Created [https://dns.googleapis.com/dns/v1/projects/topse-nagaku/managedZones/topse-private-zone/changes/3].
ID  START_TIME                STATUS
3   2020-02-15T15:15:03.329Z  pending
Deleted [https://dns.googleapis.com/dns/v1beta2/projects/topse-nagaku/managedZones/topse-private-zone].


## FileStore 削除

In [6]:
!gcloud beta filestore instances delete {res_info['filestore_name']} --zone {res_info['zones'][0]} --quiet

Waiting for [operation-1581779706876-59e9ec98a9119-086439b0-1d63da5d] to finish
...done.                                                                       


## CloudSQL 削除

In [7]:
!gcloud sql instances delete {res_info['cloud_sql_name']} --quiet

Deleting Cloud SQL instance...done.                                            
Deleted [https://sqladmin.googleapis.com/sql/v1beta4/projects/topse-nagaku/instances/topse-wordpress-mysql-20200215112636].


## VPC Peering削除

In [8]:
!gcloud compute addresses delete {res_info['vpc_peering_name']} --global --quiet

Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/addresses/topse-vpc-peering].


## GKE削除

In [9]:
!gcloud container clusters delete {res_info['cluster_name']} --region {res_info['region']} --quiet

Deleting cluster topse-cluster...done.                                         
Deleted [https://container.googleapis.com/v1/projects/topse-nagaku/zones/us-west1/clusters/topse-cluster].


## ファイアーウォールルール削除

In [10]:
!gcloud compute firewall-rules delete {res_info['vpc_name']}-ssh-ingress --quiet
!gcloud compute firewall-rules delete {res_info['vpc_name']}-mysql-ingress --quiet

Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/firewalls/topse-vpc-ssh-ingress].
Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/firewalls/topse-vpc-mysql-ingress].


## サブネット削除

In [11]:
!gcloud compute networks subnets delete {res_info['subnet_name']} --region={res_info['region']} --quiet

Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/regions/us-west1/subnetworks/topse-subnet].


## VPC削除

In [12]:
!gcloud compute networks delete {res_info['vpc_name']} --quiet

Deleted [https://www.googleapis.com/compute/v1/projects/topse-nagaku/global/networks/topse-vpc].


## GCP CLI 認証情報削除

In [13]:
MYNAME = !whoami
SA_KEYFILENAME = "topse-nagaku-e6bb79af7280.json"

認証情報を削除

In [14]:
!gcloud auth revoke 'topse-students@topse-nagaku.iam.gserviceaccount.com'

Revoked credentials:
 - topse-students@topse-nagaku.iam.gserviceaccount.com
                  Credentialed Accounts
ACTIVE  ACCOUNT
*       226115571655-compute@developer.gserviceaccount.com


configration削除

In [15]:
!gcloud config configurations activate default

Activated [default].


In [16]:
!gcloud config configurations delete {MYNAME[0]} --quiet

The following configurations will be deleted:
 - user4
Deleted [user4].


サービスアカウントの認証キーファイルを削除する

In [17]:
!rm -f {SA_KEYFILENAME}

In [18]:
!date

Sat Feb 15 15:24:19 UTC 2020
